In [1]:
import pandas as pd

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

## 05. GBM(Gradient Boosting Machine)

### 부스팅 알고리즘
- 여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치를 부여하며 오류를 개선해나가는 모델
- (예) AdaBoost, Gradient Bosst

### GBM
- 랜덤포레스트 기반
- AdaBoost와 유사하나 가중치 업데이트를 <U>경사 하강법</U>을 이용  
- $error = y - F(x)$ 을 최소화하는 방향성을 가지고 반복적으로 가중치 값을 업데이트

#### 1. 장점
- 랜덤포레스트보다 더 나은 에측 성능

#### 2. 단점
- 시간이 오래 걸리고, 하이퍼 파라미터 튜닝 노력이 더 필요
- 과적합 규제(regulation)가 없음

*파이썬 라이브러리를 활용한 머신러닝 교재에 어떤 모델을 기반으로 하는지 등 더 자세하게 나와있는 것 같다! [참고링크](https://jhryu1208.github.io/data/2020/11/21/ML_bagging-extra-ada/)*

In [4]:
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
# WARNING : this code takes so long
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()

gbm = GradientBoostingClassifier(random_state=0)
gbm.fit(X_train, y_train)
pred = gbm.predict(X_test)

In [6]:
print("accuracy : {:.4f}".format(accuracy_score(y_test, pred)))
print("time : {:.1f}".format(time.time()-start_time))

accuracy : 0.9393
time : 579.7


### GBM 하이퍼 파라미터 튜닝
```
class sklearn.ensemble.GradientBoostingClassifier(*, loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)[source]
```

- `loss `(default='deviation) : 손실함수, 특별한 이유가 없으면 그대로 적용 ('exponential'=AdaBoost algorithm??[링크](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html))
- `learning_rate`(default=0.1) : weak learner가 순차적으로 오류 값을 보정해 나가는데 적용하는 계수, (0~1), n_estimator와 상호 보완적으로 조합해 사용
- `n_estimator`(default=100) : weak learner의 갯수, 갯수가 많을수록 시간↑
- `subsample`(default=1) : weak learner가 학습에 사용하는 데이터 샘플링 비율

In [ ]:
from sklearn.model_selection import GridSearchCV

start_time = time.time()
params = {
    "n_estimators":[100,500],
    "learning_rate":[0.05,0.1]
}

gbm = GradientBoostingClassifier()
grid_gbm = GridSearchCV(gbm, param_grid=params, cv=2, verbose=1)
grid_gbm.fit(X_train, y_train)

print("best hyper-params : ", grid_gbm.best_params_)
print("best score : ", grid_gbm.best_score_)
print("time : ", time.time()- start_time)

In [ ]:
pred = grid_gbm.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred)
print("acc : {:.4f}".format( acc))